# EDA and Modelling Notebook

## Data Understanding

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import data_preparation_functions as dp
import figure_functions as fg
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.stats.stattools import jarque_bera
import math

plt.style.use('bmh')
%matplotlib inline

In [ ]:
# Load Kings county database and preview first few entries
kc_df = pd.read_csv('data/kc_house_data.csv')
kc_df.head()

In [ ]:
# Taking a look at all the columns, data types, and missing entries
kc_df.info()

The loaded data contains 21,597 entries, 21 columns, 9 of type int64, 3 of type float64, and 6 of type object. It contains:

- `id` column which have a unique id value for each house
- `date` column with the sale date (Years 2014 - 2015)
- `price` column with the price of sale and our target variable
    - <p>mean: \$540296</p>
    - <p>std: \$367368</p>
- `bedrooms`, `bathrooms` and `floors` columns with counts of each in their respective house
- four area variables, (`sqft_living`, `sqft_lot`, `sqft_above`, `sqft_basement`), defining the square footage of living space, lot space, area of house apart from the basement, and area of the basement.
- two area variables, (`sqft_living15`, `sqft_lot15`), describing the living and lot area of the 15 nearest neighbors
- `waterfront` variable of whether the house is on a waterfront (Yes/No)
- `view` describing quality of view from house
    - (None, Fair, Average, Good, Excellent)
- `condition` for the overall maintanence condition of the house
    - (Poor, Fair, Average, Good, Very Good)
- `grade` describing overall construction and design grade of the house 
    - (Poor, Low, Fair, Low Average, Average, Good, Better, Very Good, Excellent, Luxury, Mansion)
- `yr_built` for when the house was built
- `yr_renovated` for when house was renovated last (if applicable)
- three location columns detailing `zipcode`, latitude (`lat`) and longitude (`long`) of the house.
    - Data consists of an area covering 70 zipcodes


In [ ]:
# Checking the years spanned by the data set
sales_year = kc_df.date.map(lambda x: int(x[-4:]))

print('The data spans the years {} to {}'.format(min(sales_year), max(sales_year)))

# finds mean and standard deviation of price
print('Sales price mean: ${}, stdev: ${}'.format(int(kc_df.price.mean()),int(kc_df.price.std())))

In [ ]:
# Checking view values
kc_df.view.value_counts()

In [ ]:
# Checking condition values
kc_df.condition.value_counts()

In [ ]:
# Checking grade values
kc_df.grade.value_counts()

In [ ]:
# Checking how many unique zipcodes are included in the dataset
num_of_zip = len(kc_df.zipcode.unique())
print('There are {} unique zipcodes in this data set'.format(num_of_zip))

## Data Preparation


At first glance, the columns for `id`, latitude (`lat`), and longitude (`long`) can be removed. The `id` column is good for reference, but not good for a linear regression model. The latitude and longitude appear to be difficult variables to deal with since they describe an exact location. In a linear regression model their seperate coefficients would only be good for how far in a cordinal direction they are. Having zones like a zipcode would be preferable as that defines a region, though having 70 unique zipcodes in our data set would create a lot of columns if they were treated categorically and one hot encoded. On the other hand treating them as continuous data doesn't make sense as they are not in an order with any statistical significance. Therefore the `zipcode` column will also be ommitted from our calculations. Another column that can be removed is the `date`. If our data set spanned a longer time this column may be used to adjust for inflation, or observe outside conditions that caused housing prices to drastically change like the 2006 housing bubble. Since our data only spans the years of 2014 and 2015 this isn't necessary.

We also have to consider which columns can actually pertain to our problem. We are looking for the best variables to improve to increase the sales price. Variables like `waterfront`, `sqft_lot`, `view`, `yr_built`, `yr_renovated`, `sqft_basement`, `sqft_living15` and `sqft_lot15` are variables that depend on location and are impossible or very hard to change. Considering this these variables will also be ommitted from our calculations.

Another thing to consider is correlation between features. Looking at the feature descriptions `sqft_living` and `sqft_above` may be very similar. Let's check their correlation.

In [ ]:
kc_df[['sqft_living', 'sqft_above']].corr().iloc[0, 1]

There's some clear correlation between them, and seeing as most living space is above the basement this most likely isn't spurious correlation. Therefore one of the variables should be dropped from the analysis. For this analysis `sqft_above` will be dropped.

In [ ]:
# Dropping unnecessary columns from original dataset
unn_columns = ['id', 'date', 'lat', 'long', 'zipcode', 'waterfront',
               'view', 'yr_built', 'yr_renovated', 'sqft_living15',
              'sqft_lot15', 'sqft_lot', 'sqft_basement', 'sqft_above']

kc_df_iprep = kc_df.drop(columns=unn_columns, axis=1).copy()
kc_df_iprep.info()

We now need to handle the object columns. 

The `grade` columns values already have a numerical ranking assigned to them followed by the description. Therefore we will just remove the description for each.

The `condition` column only contains the description, so they will be replaced using the map below.

- Poor: 1
- Fair: 2
- Average: 3
- Good: 4
- Very Good: 5

The mapping data will be saved for later reference.

In [ ]:
# Prepares columns for analysis
kc_df_iprep, condition_map, grade_map = dp.initial_prep(kc_df_iprep)
kc_df_iprep.head()

Let's look at the data to check for any outliers in the data.

In [ ]:
# Plots histograms for each variable
kc_df_iprep.hist(figsize=(16,16), bins=30);

There appears to be some outliers in the `bedrooms`, `sqft_living` and `price` columns. We will call outliers in `sqft_living` and `price` anything with values above 8,000 and 4,000,000 respectively. Let's take a look at bedroom values. 

In [ ]:
# Checks bedrooms value counts
kc_df_iprep.bedrooms.value_counts()

There's one entry of 33 bedrooms that is much larger than the rest that will be ommitted.

In [ ]:
# Checks the percentage of outliers contained in our dataset determined by defined values
len_of_outliers = len(kc_df_iprep[(kc_df_iprep.sqft_living > 8000)
                              | (kc_df_iprep.price > 4000000)
                              | (kc_df_iprep.bedrooms == 33)])
print('Percentage of outliers: {}%'.format(round(100*(len_of_outliers/len(kc_df_iprep)), 4)))

These outliers only account for about .07% of the data, thus all will be omitted. While ommitting these entries, duplicates will also be checked for and dropped.

In [ ]:
# Drops outliers and duplicate entries
kc_df_iprep = dp.omit_outliers_dups(kc_df_iprep)

# Saves the initial prep df to pickle file
kc_df_iprep.to_pickle('data/init_df.pk1')

In [ ]:
# Plots histograms for each variable
kc_df_iprep.hist(figsize=(16,16), bins=30);

Though the columns for `bedrooms`, `bathrooms`, `condition` and `grade` look like categories that can possible be One Hot Encoded, it will be difficult to make a suggestion based on the individual coefficients for each subcategory. Thus they will be treated as continuous features, they have already been encoded with appropriate numerical values.

With the data prepped let's take a quick look at some basic statistics.

In [ ]:
kc_df_iprep.shape

Our prepared dataset has 21,480 entries with 7 features.

In [ ]:
kc_df_iprep.describe()

One thing that stands out is the large standard deviations for `price` compared to its mean. The quartiles look to be in decent range to the mean, though the max value is much larger. This indicates that it might have a pretty heavy tail. 

With the initial prep done, let's split the dataset into a training and test set to later see if our models properly represent the data.

In [ ]:
# Target Variable
y = kc_df_iprep.price.copy()

# independent variables
X = kc_df_iprep.drop(columns='price', axis=1).copy()

# Splits the data into two sets at a 4:1 ratio of variables for train:test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)

## Modeling

The initial data preparation is completed, it's time to start our first model which will be a simple linear regression of the variable with the highest correlation to price.

### Base Model


In [ ]:
# Checks correlation of all variables to price
pd.concat([X_train, y_train], axis=1).corr().price.sort_values(ascending=False)

The variable with the highest correlation to `price` is `sqft_living`. Let's take a look at the scatter plot and histograms for the variables.

In [ ]:
# Plots figures compairing price and sqft_living
fg.base_plots(X_train.sqft_living, y_train)

The histograms for square foot living and price show that the data is a bit skewed to the left. If our model performs poorly maybe we can check if the log of these variable will perform better. Our analysis cross validates all variables using a shuffle split that creates ten random samples of training and test sets from our training set. The scores reported below are the average R<sup>2</sup> values for both sets.

In [ ]:
# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train[['sqft_living']], y_train, 'base_model')
df_rsquared_cv

The resulting R<sup>2</sup> score has our modeling explaining only 48% of of our target variables variance. The Train and Test scores are also very close. This shows that this model is underfit with a large bias and low variance. Let's take a quick look at the logs of these variables and the resulting model.

In [ ]:
# Plots the log transformed figures
fg.base_plots_log(X_train.sqft_living, y_train)

It appears that the log of both variables is very similar to a normal distribution, and results in a more linear relationship between price and square foot living. Let's now test the linear regression model created from these variables.

In [ ]:
# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(np.log(X_train[['sqft_living']]), np.log(y_train),
                   'log_base_model', df_rsquared_cv)
df_rsquared_cv

The R<sup>2</sup> scores have actually gone down compared to our original model even though there looks to be a more linear relationship, only explaining around 45% or our target variables variance. Therefore the previous model will be kept as our baseline model. 

### Model 2 - With all Features

Since the base model has a high bias and low variance, let's make it more complicated by adding all the features to the model and check the results.

In [ ]:
# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train, y_train, 'model_2', df_rsquared_cv)
df_rsquared_cv

There is some clear improvement in the R<sup>2</sup> values of our second model. This model now explains around 55% of our target variables variance. While we did improve our bias by a decent amount, it is still pretty high. Our training and test scores are also still very close so our variance is low. This means we still have a pretty underfit model. Let's take a look at the scatter plots for all of the variables to check for linearity.

In [ ]:
# plots seperate scatter plots of price vs all independent variables
fg.all_scatters(X_train, y_train)

There are a few features that don't have a very linear relationship with our target variable `price`, which breaks one assumption of linear regression. Let's take a look at the resulting coefficients and other scores resulting from this model.

In [ ]:
# Creates model using statsmodels, and prints summary
model2 = dp.model_summary(X_train, y_train)
model2.summary()

Looking at the summary the three features with the most impact on price are `grade`, `condition` and `sqft_living`. Something that also stands out is the Jarque-Bera score which in a residual normality check. One of assumptions of linear regression is that the residuals are normally distributed. The extremely high score of 153,046 shows that this isn't true for this model.

### Model 3 - Check for Multicollinearity

Looking at the columns there may be some the are collinear, let's check the dataset for multicollinearity and adjust the model as needed.

In [ ]:
# Prints top pairs with highest correlation
feature_corr = dp.correlation_check(X_train)
feature_corr

It appears the column `sqft_living` has high correlation with a lot of features. This makes sense since bedrooms and bathrooms are a part of the living space in a house. One can also argue that during renovations the most common additions are bedrooms and bathrooms. Therefore this next model will remove the `sqft_living` feature.

In [ ]:
# Creates training set for model 3
X_train_model3 = X_train.drop(columns='sqft_living', axis=1)

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model3, y_train, 
                                'model_3', df_rsquared_cv)
df_rsquared_cv

In [ ]:
# Creates model using statsmodels, and prints summary
model3 = dp.model_summary(X_train_model3, y_train)
model3.summary()

This model has worse R<sup>2</sup> scores than the previous model, explaining only 49% of our target variables variance. The Jarque-Bera score is still extremely high as well at around 178,000.

### Model 4 - Multiple Correlation Fix

Removing the feature `sqft_living` because of multicollinearity didn't improve our model, this may be because it also has the most linear relationship with our target variable. For this model let's try removing some features that have high collinearity with `sqft_living` instead. These features will include `bedrooms` and `bathrooms`. 

In [ ]:
# Creates independent training data used for model 4
X_train_model4 = X_train.drop(columns=['bedrooms', 'bathrooms'], axis=1)

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model4, y_train, 
                                'model_4', df_rsquared_cv)
df_rsquared_cv

In [ ]:
# Creates model using statsmodels, and prints summary
model4 = dp.model_summary(X_train_model4, y_train)
model4.summary()

This model performed slightly worse than model two with a comparable Jarque-Bera score and an R<sup>2</sup> score of .547 compared to .554. Though one could argue that this may be the prefered model considerer it is "simpler" than model two by having less features.

### Model 5

Since the fourth model has arguably been the best model to use so far, it's residuals will be examined to determine if there are outliers causing the lack of normality.

Let's check the residual distribution.

In [ ]:
sns.displot(model4.resid, kde=True);
plt.title('Residuals');

The residual plot clearly shows a long tail on the right side. Let's check if any features are associated with the residuals above 100,000 since the left tail ends at around -100,000 and the center is pretty close to 0.

In [ ]:
fg.high_resid_plots(X_train_model4, y_train, model4, 1e6)

It appears that the entries with high residuals are spread out for the features `floors` and `condition`. For the other features they may only cover a portion of the feature. Let's check what percentage of the dataset is above the lowest value for the features `sqft_living`, `grade` and `price`.

In [ ]:
columns = ['sqft_living', 'grade', 'price']
dp.outlier_percentage(X_train_model4, y_train, model4, 1e6, columns)

The percentage of entries above the lowest value of the high residual entries for `price` is only around %2.5. The percentage for features `sqft_living` and `grade` are extremely high, so for this model let's try taking out the entries depending on `price`.

In [ ]:
model5_sets = dp.model5_data(X_train_model4, y_train, 
                             X_test.drop(columns=['bedrooms', 'bathrooms'], 
                                         axis=1), y_test, model4, 
                             1e6, 'price')
print('Price Cutoff = ${}'.format(model5_sets['cutoff']))

With a cutoff price being set, this model can only be used for houses with a sales price lower than 1.5 million dollars. 

Using this new training set, we can now build our model.

In [ ]:
# Grabs training set data
X_train_model5 = model5_sets['X_train']
y_train_cutoff = model5_sets['y_train']

# Cross validates training data from fifth model
model5_results = dp.cross_val(X_train_model5, y_train_cutoff)

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model5, y_train_cutoff, 
                                'model_5', df_rsquared_cv)
df_rsquared_cv

In [ ]:
model5 = dp.model_summary(X_train_model5, y_train_cutoff)
model5.summary()

The resulting model has a lower R<sup>2</sup> score of around .521 compared to .547 of the fourth model. The Jarcque-Bera score is still pretty high, though compared to model 4 it has greatly improved. One thing to note though is the large condition number that may be caused by the large P value for feature `floors`. Let's try a model without the floors column.

### Model 6

In [ ]:
# Creates independent training data used for model 6
X_train_model6 = X_train_model5.drop(columns='floors', axis=1)

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model6, y_train_cutoff, 
                                'model_6', df_rsquared_cv)
df_rsquared_cv

In [ ]:
model6 = dp.model_summary(X_train_model6, y_train_cutoff)
model6.summary()

Removing the `floor` feature didn't improve the condition number, this brings to light that it may be a scaling issue since `sqft_living` has a much higher value than those of the other features. Let's try scaling the `sqft_living` feature down by 1000.

### Model 7

In [ ]:
# Creates independent training data used for model 7
X_train_model7 = X_train_model6.copy()
X_train_model7.sqft_living = X_train_model7.sqft_living/1000

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model7, y_train_cutoff, 
                                'model_7', df_rsquared_cv)
df_rsquared_cv

In [ ]:
model7 = dp.model_summary(X_train_model7, y_train_cutoff)
model7.summary()

Scaling the feature `sqft_living` has solved the high condition number issue. Let's try to validate this as the final model with it's R<sup>2</sup> score of around .521, relatively low Jarque-Bera score, simplicity and low condition number.  

### Model 7 Validation

Linear regression models need to fulfill four assumptions.
<ol>
    <li>Linearity</li>
    <li>Residual Normality</li>
    <li>Homoskedasticity</li>
    <li>Features are Linearly Independant</li>
</ol>

#### Linearity

In [ ]:
# plots seperate scatter plots of price vs all features
fg.all_scatters(X_train_model7, y_train_cutoff)

The data for `sqft_living` and `grade` cones out but sees to be following a linear direction. The `condition` plot doesn't appear linear at all. Let's check a quick density plot to get a better idea of its trend.

In [ ]:
sns.kdeplot(x=X_train_model7.condition, y=y_train_cutoff, fill=True);
plt.title('Condition vs. Price Density Plot');

While the price range at each condition level fans pretty wide, it appears the the densest regions of each are in a linear trend.

#### Normality

In [ ]:
fg.normality_plots(model7.resid)

In [ ]:
jb = jarque_bera(model7.resid)
print('Jarque Bera Score: {}'.format(round(jb[0], 2)))
print('Jarque Bera P-value: {}'.format(round(jb[1], 4)))

The plots show a distribution that is skewed to the left, with a short left tail, and a thicker longer right tail. The QQ-plot confirms that the distribution is skewed. The Jarque Bera score is still pretty high, though it is relatively low compared to the other models attempted in this notebook. As shown before, to improve this the log function of the target variable and some features must be taken. While this helped with making the residuals have a more normal distribution it made the model more complicated to understand and use.

#### Homoskedasticity

In [ ]:
fg.homoskedasticity_plot(y_train_cutoff, model7)

The plot above shows a scatter plot of the residuals against the expected outcome. In order for the model to be homoscedastic the residual variance needs to be continuous throughout the whole range of outcomes. This plot clearly shows that this model is not homoskedastic by curving upward as the expected value increases.

#### Multicolinearity

In [ ]:
# Prints top pairs with highest correlation
model7_corr = dp.correlation_check(X_train_model7)
model7_corr

There still appears to be some multicollinearity between `grade` and `log_sqft_living`. The feature `grade` was defined as the overall construction and design of the house. This means while the size can alter the grade, it is also dependant on the interior design of the house. Therefore this may be spurious correlation and kept in the model. 

This model has failed the assumptions of homoskedasticity and residual normality, while also arguably failing linearity. Thus more modeling has to be completed. 

### Model 8 - Log Features

As seen before `sqft_living` and `price` appear to have a logarithmic form. Let's try a model by taking the log of these variables. Model 4 will be used as the base model before being log transformed. Model 4 included data with prices greater than 1.5 million dollars, and is unscaled.

In [ ]:
# Creates independent training data used for model 8
X_train_model8 = dp.log_columns(X_train_model4, columns='sqft_living')
y_train_log = dp.log_columns(y_train)

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model8, y_train_log, 
                                'model_8', df_rsquared_cv)
df_rsquared_cv

In [ ]:
# Creates model using statsmodels, and prints summary
model8 = dp.model_summary(X_train_model8, y_train_log)
model8.summary()

Taking the log of out target variable `sales_price` and feature `sqft_living` slightly improves the models R<sup>2</sup> score while significantly improving the Jarque-Bera score to 65.6. One thing to point out again is the high P-value of the feature `floors`. Let's try a new model without this feature.

### Model 9

In [ ]:
# Creates independent training data used for model 9
X_train_model9 = X_train_model8.drop(columns='floors', axis=1)

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model9, y_train_log, 
                                'model_9', df_rsquared_cv)
df_rsquared_cv

In [ ]:
# Creates model using statsmodels, and prints summary
model9 = dp.model_summary(X_train_model9, y_train_log)
model9.summary()

Let's try and validate this model, since it has a relatively good R<sup>2</sup> score of .559, a good Jarque-Bera of 63.6, and condition numbers of 242.

### Model 9 Validation

#### Linearity

In [ ]:
# plots seperate scatter plots of price vs all features
fg.all_scatters(X_train_model9, y_train_log)

The figures containing features `log_sqft_living` and `grade` are clearly much more linear than the previous attempt at validating model 7. Though the figure for `condition` is on the border. Once again lets check the density plot.

In [ ]:
sns.kdeplot(x=X_train_model9.condition, y=y_train_log, fill=True);
plt.title('Condition vs. Price Density Plot');

The densest parts are more centered and appear to be more linearly alligned than with model 7. After looking at these figure I can comfortably say our variables are linear.

#### Normality

In [ ]:
fg.normality_plots(model9.resid)

Though the models residual distribution isn't perfectly normal with it's short tails. It appears to be more center and is certainly an improvement to model 7.

#### Homoskedasticity

In [ ]:
fg.homoskedasticity_plot(y_train_log, model9)

This model has still not solved the heteroskedastic problem.

#### Multicolinearity

In [ ]:
# Prints top pairs with highest correlation
model9_corr = dp.correlation_check(X_train_model9)
model9_corr

This model has a similar relationship between `log_sqft_living` and `grade` as model 7.

This model still fails some of the four assumptions. Let's revert back to using non-log transformed variables. Back in model 7 we only scaled the `sqft_living` column. For this next model lets scale all independant variables to have min and max values of 0 and 1.

### Model 10

In [ ]:
# Creates independent training data used for model 10
X_train_model10 = X_train_model6.copy()

X_train_model10_scaled = dp.min_max_scaler(X_train_model10)
# y_train_model10_scaled = dp.min_max_scaler(y_train_cutoff)

# Returns dataframe consisting models cross validated average Rsquared scores
df_rsquared_cv = dp.rsquared_df(X_train_model10_scaled, 
                                y_train_cutoff, 'model_10', 
                                df_rsquared_cv)
df_rsquared_cv

In [ ]:
# Creates model using statsmodels, and prints summary
model10 = dp.model_summary(X_train_model10_scaled, y_train_cutoff)
model10.summary()

This model has a similar Jarque-Bera score to model 7, though has an improved condition number of 20.1. Let's take a look if it fulfills the assumptions.

#### Linearity

In [ ]:
# plots seperate scatter plots of price vs all features
fg.all_scatters(X_train_model10_scaled, y_train_cutoff)

#### Normality

In [ ]:
fg.normality_plots(model10.resid)

#### Homoscedasticity

In [ ]:
fg.homoskedasticity_plot(y_train_cutoff, model10)

#### Multicollinearity

In [ ]:
# Prints top pairs with highest correlation
model10_corr = dp.correlation_check(X_train_model10_scaled)
model10_corr

This model performed similarly to model 7, of which we declared failed the homoscedasticity test and was on the border of have having a normal residual spread. Though after working through multiple models, this appears the be the best one in regards to homoscedasticity, condition number and simplicity. Therefore this will be our final model.  

In [ ]:
X_train_final = X_train_model10_scaled
y_train_final = y_train_cutoff

#### Root Mean Square Percent Error (RMSPE)

The error measure used for this test will utilize percentage. This decision was made because the model is used to determine a change of currency.  

In [ ]:
y_test_final = model5_sets['y_test']
X_test_final = model5_sets['X_test'].drop(columns='floors', axis=1)

X_test_final = dp.min_max_scaler(X_test_final, X_train_model10, test=True)

In [ ]:
# Saves final test and training sets as pickle files
X_train_final.to_pickle('data/X_train_final.pk1')
y_train_final.to_pickle('data/y_train_final.pk1')
X_test_final.to_pickle('data/X_test_final.pk1')
y_test_final.to_pickle('data/y_test_final.pk1')

In [ ]:
train_rmspe = 100 * dp.RMSPE(X_train_model10_scaled, y_train_cutoff, model10)
test_rmspe = 100 * dp.RMSPE(X_test_final, y_test_final, model10)


print('Training split RMSPE: {}%'.format(round(train_rmspe, 2)))
print('Test split RMSPE: {}%'.format(round(test_rmspe, 2)))

The RMSPE for the training and test splits are very close to eachother thus proving the model isn't overfit. The error of a little over 40% is definently not a great number but appears to be the best we can do for now.

#### Top Features

The goal of this model is to find the best features to improve in order to increase the home value. Thus let's find the top two features with this target in mind.

In [ ]:
model10.params

The top two features from our model have the two highest coefficients. These turn out to be the `sqft_living` and `grade` features.

#### Interpretting the Model

Since we scaled the independant features to create the model, in order to interpret it in original units we must find the rate of change. The scaling method used was linear so we just need to find the slope. The equation used to scale the features is shown below, $x_{i}$ being the values in the feature, $x_{min}$ and $x_{max}$ being the min and max values respectively of the feature being described by values of $x$. 
<br><br>
<center>$$x_{i scaled} = \displaystyle \frac{x_{i} - x_{min}}{x_{max} - x_{min}}$$</center>
<br><br>
In the above equation the values for $x_{min}$ and $x_{max}$ are constant for all values of $x$ in each individual feature. Therefore it can be determined that the rate of change for each feature is defined by the feature coefficient divided by $(x_{max} - x_{min})$.
<br><br>

In [ ]:
rates = dp.get_rates(model10, X_train_model10)
rates.to_pickle('data/rates.pk1')

In [ ]:
print('The rate of change per sqft of living space is ${}'
      .format(round(rates.loc['sqft_living'], 2)))
print('The rate of change per change in grade is ${}'
      .format(round(rates.loc['grade'], 2)))

## Conclusion

This models goal was to find the features that had the largest affect on changing the price of a home. The approach was to only include variables that were deemed changeable, then build our model using these features. The final model has a Root Mean Squared Percent Error (RMSPE) of around 41% and an R<sup>2</sup> score of .521 therefore explaining around 52% of our target variables variance. The top two features were determined to be `sqft_living` and `grade`. A change of one `sqft_living` unit equates to a \$104.63 increase in price. While a single improvement in `grade` equates to a change of \$96,872 in price. Therefore it is suggested to either increase the square footage of the living space, or to remodel the living space to increase the grade of the home. Though before taking any action an analysis of material and labor costs should be taken into consideration. Since this model had a high RMSPE and didn't pass all the assumptions, it is hard to recommend its use with full confidence. 